In [28]:
import objgraph 
import pygame
from Projectile import Projectile, Bomb
from Enemy import Enemy, EnemyFactory
from Player import Player
from  Weapon import WeaponFactory
from Weapon import Weapon
import gc

In [29]:
class NoFlyWeightEnemy():
    projectiles = pygame.sprite.Group()
    def __init__(self, name: str, image: pygame.Surface, weapon_name: str, speed: float, 
                 default_health: int, pos: tuple[float, float]):
        super().__init__()
        self.image = pygame.transform.scale(pygame.image.load("./Images/smallEnemy.png"), (50,50))
        self.rect = self.image.get_rect()
        self.speed = speed
        self.weapon = WeaponFactory.get(weapon_name)

        self.name = name
        self.health = default_health
        self.movement_vector = [0, 0]
        self.pos = list(pos)
        self.last_shot_time = pygame.time.get_ticks()
        
    def move(self, enemies: pygame.sprite.Group, player_pos: tuple[float, float], tDelta: float):
        self.movement_vector = (player_pos[0] - self.pos[0],
                               player_pos[1] - self.pos[1])
        self.movement_vector = Weapon.normalize_vector(self.movementVector)
        self.pos[0] += self.movement_vector[0] * self.speed * tDelta
        self.pos[1] += self.movement_vector[1] * self.speed * tDelta
        
        # Collision test with other enemies
        self.movementVector = [0, 0]
        for sprite in enemies:
            if sprite is self:
                continue
            if pygame.sprite.collide_circle(self, sprite):
                self.movement_vector[0] += self.pos[0] - sprite.pos[0]
                self.movement_vector[1] += self.pos[1] - sprite.pos[1]

        self.movement_vector = Weapon.normalize_vector(self.movement_vector)
        self.pos[0] += self.movement_vector[0] * 0.5  # The constant is how far the sprite will be
        self.pos[1] += self.movement_vector[1] * 0.5  # dragged from the sprite it collided with
        
        self.rect.topleft = self.pos

    def attack(self, target_pos):
        self.weapon.attack(self, target_pos, self.last_shot_time)
        self.last_shot_time = pygame.time.get_ticks()

    def render(self, surface):
        surface.blit(self.image, self.pos)
    

In [30]:
enemyFlyweight = EnemyFactory.get("small")
enemy = Enemy(name = "small", pos=(0,0))
no_flyweight_enemy = NoFlyWeightEnemy(name="small", image=pygame.image.load("./Images/smallEnemy.png"), weapon_name="melee", speed=1, default_health=1, pos=(0,0))

gc.collect()
objgraph.show_refs([enemy, no_flyweight_enemy], filename='memory_graph.png')

AttributeError: 'NoneType' object has no attribute 'image'